In [1]:
!pip install torch torchvision gdown -q

In [2]:
import gdown
import os
from PIL import Image
import gc

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import average_precision_score


import random
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

### Multi-class probplem

In [14]:
gdown.download("https://drive.google.com/file/d/1q8Jc3LyQfNxuzAAvIQFAcwrgliHRijix/view?usp=share_link","./class_data.zip", quiet=False, fuzzy=True)
!unzip -q ./class_data.zip

### Resnet training

In [3]:
if not os.path.exists('multi_class_output'):
    os.mkdir('multi_class_output')
else:
    print('multi_class_output exists')

In [8]:
!nvidia-smi

Sun May 21 19:02:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:41:00.0 Off |                  N/A |
|  0%   40C    P2   149W / 370W |   7166MiB / 24576MiB |     27%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:61:00.0 Off |                  N/A |
|  0%   

In [5]:
def clean():
    torch.cuda.empty_cache()
    gc.collect()

In [6]:
clean()

In [9]:
os.environ['CUDA_VISIABLE_DEVICES']='0'

In [10]:
device = torch.device('cuda')

### VIT

In [36]:
from transformer.vit import Transformer
from utils.trainer import Trainer
from data.multi_class_build_data import build_dataloader

BATCH_SIZE=8

transfrom = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS = build_dataloader(transfrom, BATCH_SIZE)

In [15]:
criteriation=nn.CrossEntropyLoss()

In [30]:
transformer = Transformer(img_size=(240, 320),
                          patch_size=(8, 8),
                          in_channels=3,
                          n_classes=NUM_CLASS,
                          embed_dim=128,
                          depth=6,
                          n_heads=16,
                          mlp_ratio=4.,
                          qkv_bias=True,
                          p=0.3,
                          attn_p=0.3
                         )

In [31]:
transformer = transformer.to(device)

In [32]:
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.001)

### Training 

In [37]:
NAME = 'vit'

trainer = Trainer(transformer , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=2,
                  path_output='multi_class_output/{name}.pt'.format(name=NAME),
                  multi_label=False
                 )

In [38]:
trainer.training()

[1]/[2] Epoch starts
	 Batch train loss: 1.2611844539642334, accuracy 0.5
	 Batch train loss: 1.3615361452102661, accuracy 0.25
	 Batch train loss: 1.3960784673690796, accuracy 0.25
	 Batch train loss: 1.382154107093811, accuracy 0.375
	 Batch train loss: 1.396828055381775, accuracy 0.125
[1]/[2] End epoch: train loss: 1.388670197976597, val loss: 1.4063543627240067
	 Epoch train accuracy: 0.25851160287857056, val accuracy: 0.24929634097305992

[2]/[2] Epoch starts
	 Batch train loss: 1.4980098009109497, accuracy 0.125
	 Batch train loss: 1.4070000648498535, accuracy 0.125
	 Batch train loss: 1.1671994924545288, accuracy 0.625
	 Batch train loss: 1.50556480884552, accuracy 0.25
	 Batch train loss: 1.5396944284439087, accuracy 0.25
[2]/[2] End epoch: train loss: 1.3361544116045967, val loss: 1.3421580781407465
	 Epoch train accuracy: 0.3368484377861023, val accuracy: 0.3606755126658625



### Evaluate model

In [39]:
# save cuda memory
# culd skip

del transformer


transformer = Transformer(img_size=(240, 320),
                          patch_size=(8, 8),
                          in_channels=3,
                          n_classes=NUM_CLASS,
                          embed_dim=128,
                          depth=6,
                          n_heads=16,
                          mlp_ratio=4.,
                          qkv_bias=True,
                          p=0.3,
                          attn_p=0.3
                         )

In [40]:
transformer.load_state_dict(torch.load(f'multi_class_output/vit.pt')['model_state_dict'])
transformer = transformer.to(device)

In [41]:
_, _, preds = trainer.val()

In [51]:
prediciton = pd.DataFrame([preds[0].cpu().numpy(), preds[1].cpu().numpy()]).T
prediciton.rename = ['labels', 'predictions']

prediciton.to_csv('multi_class_output/{name}.csv'.format(name=NAME), index=False)

### Multi-label problem

import pandas as pd

In [53]:
gdown.download("https://drive.google.com/file/d/15aX9gKeSlGoUJBHAQGqkL5euq0kDKkwg/view?usp=share_link","./label_data.zip", quiet=False, fuzzy=True)
!unzip -q ./label_data.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=15aX9gKeSlGoUJBHAQGqkL5euq0kDKkwg
From (redirected): https://drive.google.com/uc?id=15aX9gKeSlGoUJBHAQGqkL5euq0kDKkwg&confirm=t&uuid=e2edee54-c278-4706-a7ab-fe137a692663
To: /home/jovyan/work/label_data.zip
100%|██████████████████████████████████████████████████████████████████████████████| 4.51G/4.51G [00:52<00:00, 86.0MB/s]


replace sample/images/00000013_005.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [66]:
# Clean dataset: RUN ONLY ONCE
data = pd.read_csv('sample_labels.csv')
data['lables'] = data['Finding Labels'].str.split('|')
data['Image Index'] = './sample/images/' + data['Image Index']

chen = []
for path_img in data['Image Index'].values:
    img = Image.open(path_img)
    chen.append(transforms.ToTensor()(img).shape)

bad_img = data.loc[pd.DataFrame(chen)[0] > 1, 'Image Index'].values

for path_img in bad_img:
    img = plt.imread(path_img)
    img = img[:, :, 0]
    img = Image.fromarray(np.uint8(img * 255), 'L')
    img.save(path_img,"PNG")

In [7]:
if not os.path.exists('multi_label_output'):
    os.mkdir('multi_label_output')
else:
    print('multi_label_output exists')

### Losses

In [13]:
criteriation = nn.BCELoss()

In [5]:
from utils.trainer import Trainer
from data.multi_label_build_data import build_dataloader

BATCH_SIZE=4

transfrom = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS = build_dataloader(transfrom, BATCH_SIZE)

In [6]:
from transformer.vit import Transformer

transformer = Transformer(img_size=(256, 256),
                          patch_size=(8, 8),
                          in_channels=1,
                          n_classes=NUM_CLASS,
                          embed_dim=128,
                          depth=6,
                          n_heads=16,
                          mlp_ratio=4.,
                          qkv_bias=True,
                          p=0.3,
                          attn_p=0.3
                         )

In [62]:
transformer = transformer.to(device)

In [63]:
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-5)

In [70]:
NAME = 'vit'

trainer = Trainer(transformer , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=2,
                  path_output='multi_label_output/{name}.pt'.format(name=NAME),
                  multi_label=True
                 )

In [71]:
trainer.training()

[1]/[2] Epoch starts
	 Batch train loss: 0.29011303186416626, accuracy 0.42895299145299143
	 Batch train loss: 0.1923581212759018, accuracy 0.6666666666666666
	 Batch train loss: 0.16357667744159698, accuracy 0.8
	 Batch train loss: 0.49025410413742065, accuracy 0.5884920634920634
	 Batch train loss: 0.2088184356689453, accuracy 0.5692307692307692
[1]/[2] End epoch: train loss: 0.22505359889538656, val loss: 0.22471263094154667
	 Epoch train accuracy: 0.6901891100824052, val accuracy: 0.6822122996688117

[2]/[2] Epoch starts
	 Batch train loss: 0.10491913557052612, accuracy 1.0
	 Batch train loss: 0.15026028454303741, accuracy 0.8125
	 Batch train loss: 0.10341835767030716, accuracy 1.0
	 Batch train loss: 0.15371689200401306, accuracy 0.7857142857142857
	 Batch train loss: 0.12722177803516388, accuracy 0.8333333333333334
[2]/[2] End epoch: train loss: 0.21665471261294522, val loss: 0.220863472514127
	 Epoch train accuracy: 0.6923170739929381, val accuracy: 0.6831609621752351



### Evaluate model

In [11]:
transformer.load_state_dict(torch.load(f'multi_label_output/vit.pt')['model_state_dict'])
transformer = transformer.to(device)

In [15]:
trainer = Trainer(transformer , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  path_output='multi_label_output/{name}.pt'.format(name=NAME),
                  multi_label=True
                 )

In [16]:
_, _, preds = trainer.val()

In [84]:
pd.DataFrame(preds[0].cpu().numpy()).to_csv('multi_label_output/true_{name}.csv'.format(name=NAME), index=False)
pd.DataFrame(preds[1].cpu().numpy()).to_csv('multi_label_output/pred_{name}.csv'.format(name=NAME), index=False)